In [1]:
# gpu number setting
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = '1' ## gpu 번호 셋팅 윤건 :0, 기준 : 1, 준형 :2,

# Gpu growth setting
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)


# tensorflow & keras version check
print('tensorflow version : ' , tf.__version__)
print('keras version : ' , tf.keras.__version__)

# tensorflow gpu available check 
print('GPU available ? : ', tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))

tensorflow version :  2.3.1
keras version :  2.4.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU available ? :  True


In [2]:
import pandas as pd
import numpy as np
import pickle
import tqdm
import random
import time

from sklearn.metrics import log_loss, roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from deepctr.models import *
from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names

In [3]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [4]:
data = pd.read_csv('../../Data/movie_201130_table_3_4_5.csv')
# test = pd.read_csv('../../../ipr/data/movie_hist_test.csv')

/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# sparse_features = ['sa_id', 'album_id', 'buy_seg']
dense_features = ['buy_history_price', 'movie_meta_price', 'i30_meta_price', 'buy_tot',
                  'release_date', 'run_time']
sparse_features = list(data.columns.difference(set(dense_features)))
# target = ['label']

In [6]:
print(len(dense_features))
print(len(sparse_features))

6
58


### make unique tab

In [7]:
user_col = ['sa_id', 'buy_tot']
cont_col = list(data.columns.difference(set(user_col)))

In [8]:
user_tab = data[user_col]
cont_tab = data[cont_col]

In [9]:
user_uni_tab = user_tab.loc[list(user_tab['sa_id'].drop_duplicates().index)]
cont_uni_tab = cont_tab.loc[list(cont_tab['album_id'].drop_duplicates().index)]

In [10]:
album_id_list = list(cont_uni_tab['album_id'])

### make unique 'user', 'album' pairs

In [16]:
data_2 = data[['sa_id', 'album_id']]

In [37]:
data_2['label'] = 1

/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [40]:
tmp = time.time()
data_dict = data_2.set_index(keys=['sa_id', 'album_id']).T.to_dict('list')
print(time.time() - tmp)

2004.1762471199036


In [41]:
len(data_dict)

14474140

### negative sampling

In [42]:
def get_train_instances(train, num_negatives, album_id_list):
    '''
    설명 : train matrix를 받아서 그 안에 있는 user, item의 조합 1개당 num_negatives의 수만큼 negative sample을 만드는 함수.
    '''
    user_input, item_input, labels = [],[],[]
    
    train_keys = train.keys()
#     train_items = train.items()
    for (u, i) in tqdm.tqdm(train_keys):       
        # positive instance
        user_input.append(u)
        item_input.append(i)
        labels.append(1)
        
        # negative instances
        for _ in range(num_negatives):
            j = random.randint(0, len(album_id_list) - 1)
            album = album_id_list[j]
            while (u, album) in train_keys :
                j = random.randint(0, len(album_id_list) - 1)
                album = album_id_list[j]
            user_input.append(u)
            item_input.append(album)
            labels.append(0)
#             output_dict[(u, j)] = 0
            
        
    return user_input, item_input, labels

In [43]:
user_input, item_input, labels = get_train_instances(data_dict, 1, album_id_list)

100%|██████████| 14474140/14474140 [00:44<00:00, 328937.96it/s]


In [44]:
final_df = pd.DataFrame({'sa_id': user_input, 'album_id':item_input, 'label':labels})

In [45]:
final_df

,sa_id,album_id,label
0,205080062020,M0119C2391PPV00,1
1,205080062020,M01187B242PPV00,0
2,205090440924,M0119C2391PPV00,1
3,205090440924,M01156P227PPV00,0
4,205091021367,M0119C2391PPV00,1
...,...,...,...
28948275,500204543205,M0116AR252PPV00,0
28948276,500206206352,M0118AN125PPV00,1
28948277,500206206352,M01176U476PPV00,0
28948278,500211550347,M01143E049PPV00,1


In [46]:
merge_df = pd.merge(final_df, user_uni_tab, how='inner', on='sa_id')

In [47]:
merge_df_2 = pd.merge(merge_df, cont_uni_tab, how='inner', on='album_id')

In [48]:
merge_df_2.to_csv('../../Data/movie_201130_table_3_4_5_6.csv', index=None)

In [49]:
final_df.to_csv('../../Data/movie_201130_table_3_4_5_6_only_id_pairs.csv', index=None)